In [224]:
from mido import Message, MidiFile, MidiTrack
from collections import defaultdict, Counter
from random import uniform, randint, choice
import math

mid = MidiFile("./one_love.mid")

chords = []

seen = [-1 for _ in range(128)]
    
time = 0

longetivity = [[0] for _ in range(128)]

for track in mid.tracks:
    track = track[2:-1]
    i = 0
    while(i < len(track)-1):
        time+=track[i].time
        chord = 2 << track[i].note
        j = i+1
        if(track[j].type == "note_on"):
                seen[track[i].note] = time
        else:
            longetivity[track[i].note].append(time - seen[track[i].note])
        while(j < len(track)-1 and track[j].time == 0):
            if(track[j].type == "note_on"):
                chord += 2 << track[j].note
                seen[track[j].note] = time
            else:
                longetivity[track[j].note].append(time - seen[track[j].note])
            j += 1
        chords.append(chord)
        i = j

In [225]:
markovChain = defaultdict(list)

for i in range(len(chords[:-1])):
    markovChain[chords[i]].append(chords[i+1])

In [226]:
start = choice(list(markovChain.keys()))

In [227]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

track.append(Message('program_change', program=12, time=0))

length = 0

starts, stops = [], []

time = 0

while(length < 1500 and len(markovChain[start]) != 0):
    length+=1
    temp = start
    while(temp):
        note = int(math.log(temp & ~(temp-1), 2))
        notes.append(note)
        #track.append(Message('note_on', note=note, velocity=64, time=time))
        starts.append((note, time))
        stops.append((note, time + choice(longetivity[note])))
        temp &= temp-1
        time += 200
    start = choice(markovChain[start])

In [228]:
prev_time = 0
while(starts and stops):
    if(starts[0][1] < stops[0][1]):
        note, t = starts.pop(0)
        track.append(Message('note_on', note=note, velocity=64, time=t-prev_time))
    else:
        note, time = stops.pop(0)
        track.append(Message('note_off', note=note, velocity=64, time=t-prev_time))
    prev_time = t

In [230]:
mid.save('new_song.mid')

In [93]:
m = MidiFile("./one_love.mid")